<img src="figs/1km_Lat11_vel.gif">

# eReefs hydrodynamics cross section for GBR 1km model

## Load the required Python libraries

First of all, load the necessary libraries.

In [ ]:
import os
import numpy as np

import datetime as dt

import netCDF4
from netCDF4 import Dataset, num2date

from pylab import *
import matplotlib.ticker as mticker
import matplotlib.tri as triangle 
from matplotlib import pyplot as plt, animation

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import cmocean

import cartopy
import cartopy.crs as ccrs
cartopy.config['data_dir'] = os.getenv('CARTOPY_DIR', cartopy.config.get('data_dir'))

from IPython.display import HTML, display

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

## Define which data to be plotted.

In this section we define which data we want to read and plot.

- **inputFile**
  The netCDF input file. This can either be a downloaded file (see [How to manually download derived data from THREDDS](http://ereefs.aims.local/ereefs-aims/help/how-to-manually-download-derived-data)) or a  OPeNDAP URL from the [AIMS THREDDS server](http://thredds.ereefs.aims.gov.au/thredds/catalog.html). For this tutorial we are using the OPeNDAP URL.
- **selectedVariable**
  The name of the variable in the netCDF file.
- **selectedTimeIndex**
  The time slice in the netCDF file. Note the index starts with 0. For example, in the netCDF file, the time steps are "days". This means if you select `selectedTimeIndex=1` it refers to the second day in the netCDF file.
- **selectedDepthIndex**
  The depth slice in the netCDF file. Note the index starts with 0. See the following table for a mapping of index to value:


| Index (k) | Hydrodynamic 1km model | Hydrodynamic and BioGeoChemical 4km model |
| -: | -: | -: |
| 0 | -140.00 m | -145.00 m |
| 1 | -120.00 m | -120.00 m |
| 2 | -103.00 m | -103.00 m |
| 3 | -88.00 m | -88.00 m |
| 4 | -73.00 m | -73.00 m |
| 5 | -60.00 m | -60.00 m |
| 6 | -49.00 m | -49.00 m |
| 7 | -39.50 m |-39.50 m |
| 8 | -31.00 m | -31.00 m |
| 9 | -24.00 m | -23.75 m |
| 10 | -18.00 m | -17.75 m |
| 11 | -13.00 m | -12.75 m |
| 12 | -9.00 m | -8.80 m |
| 13 | -5.25 m | -5.55 m |
| 14 | -2.35 m | -3.00 m |
| 15 | -0.50 m | -1.50 m |
| 16 | n/a | -0.50 m |

# Connect to the OpeNDAP endpoint for a specified month. 

We query the server based on the OPeNDAP URL for a specific file "EREEFS_AIMS-CSIRO_gbr1_2.0_hydro_daily-daily--YYYY-MM-DD.nc". 


- **gbr1**: we use the Hydrodynamic 1km model and daily data for the month specified

In [ ]:
day = 1
month = 3
year = 2020

netCDF_datestr = str(year)+'-'+format(month, '02')+'-'+format(day, '02')
print('File chosen time interval:',netCDF_datestr)

inputFile = "http://thredds.ereefs.aims.gov.au/thredds/dodsC/s3://aims-ereefs-public-prod/derived/ncaggregate/ereefs/gbr1_2.0/daily-daily/EREEFS_AIMS-CSIRO_gbr1_2.0_hydro_daily-daily-"+netCDF_datestr+".nc"

We now load the dataset within the Jupyter environment...

In [ ]:
nc_data = Dataset(inputFile, 'r')

print('Get the list of variable in the file:')
print(list(nc_data.variables.keys()))

ncdata = nc_data.variables

To get information for a specific variables, we can use the following:

In [ ]:
ncdata['mean_cur']

In [ ]:
ncdata['mean_cur'].standard_name

In [ ]:
ncdata['mean_cur'].units

# Load variables

We then load the file dataset in Jupyter

In [ ]:
# Starting with the spatial domain
lat = ncdata['latitude'][:].filled(fill_value=0.)
lon = ncdata['longitude'][:].filled(fill_value=0.)

print('eReefs model spatial extent:\n')
print(' - Longitudinal extent:',lon.min(),lon.max())
print(' - Latitudinal extent:',lat.min(),lat.max())

In [ ]:
# Get time span of the dataset
time_var = ncdata['time']

# Starting time
dtime = netCDF4.num2date(time_var[0],time_var.units)
daystr = dtime.strftime('%Y-%b-%d %H:%M')
print(' - start time: ',daystr,'\n')

# Ending time
dtime = netCDF4.num2date(time_var[-1],time_var.units)
daystr = dtime.strftime('%Y-%b-%d %H:%M')
print(' - end time: ',daystr,'\n')

ntime = len(time_var)

print(' - Number of time steps',ntime,'\n')

In [ ]:
# Number of vetical points along the z-coordinate model
zc = ncdata['zc'][:].filled(fill_value=0.)
nlay = len(zc)

print('Number of vertical layers',nlay)

for k in range(nlay):
    print(f'  + vertical layer {k} is at {zc[k]} m')

# Cross section

Let's make a cross section along the latitude 11 S. 

The AIMS dataset has been resample on a regular grid, so first we find the closest point to the desired latitude. 

In [ ]:
def find_nearest(array, value):
    '''
    Find index of nearest value in a numpy array
    '''
    
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    
    return idx

We will then extract all the point along this latitude:

In [ ]:
def eReefs_cross_data(dataname, latID=None, lonID=None):
    
    '''
    This function extracts specified dataset along a latitude or longitude.
    
    args:
    - dataname: specified variable name 
    - latID: latitudinal index
    - lonID: longitudinal index
    '''
    
    # Get data
    if latID is not None:
        data = ncdata[dataname][:, :, latID, :]
    elif lonID is not None:
        data = ncdata[dataname][:, :, :, lonID]
    
    return data

Now let's start run this function...

In [ ]:
selectedLatIndex = find_nearest(lat,-11.)

selectedVariable = 'mean_cur' 
currLat = eReefs_cross_data(selectedVariable, selectedLatIndex, None)

We then define the plotting function:

In [ ]:
def plot_cross_section(tstep, xdata, ydata, xmin, xmax, dmin, dmax, color, size, fname, show=True):

    fig = plt.figure(figsize=size, facecolor='w', edgecolor='k')

    ax = plt.axes()
    plt.xlabel('Cross-section (degree)')
    plt.ylabel('Depth (m)')

    plt.xlim(xmin, xmax)
    plt.ylim(zc.min(), zc.max())

    cm = plt.pcolormesh(xdata, zc, ydata[0,:,:],  
                   cmap = color,  
                   vmin = dmin,  
                   vmax = dmax, 
                   edgecolors = 'face', 
                   shading ='flat') 

    dtime = netCDF4.num2date(time_var[0],time_var.units)
    daystr = dtime.strftime('%Y-%b-%d %H:%M')
    plt.title(ncdata[selectedVariable].short_name+', %s UTC+10' % (daystr), fontsize=11);

    # Color bar
    gca().patch.set_facecolor('1')
    cbar = fig.colorbar(cm, ax=ax, fraction=0.01, pad=0.025)
    cbar.set_label(ncdata[selectedVariable].units, rotation=90, labelpad=5, fontsize=10)
    cbar.ax.tick_params(labelsize=8)
    
    # Get z-coordinate lines
    for k in range(len(zc)):
        plt.plot([xmin, xmax],[zc[k],zc[k]],lw=0.5,c='k',alpha=0.25)
        
    
    if show:
        plt.savefig(f"{fname}_cross_time{tstep:04}.png",dpi=300, 
                bbox_inches='tight')
        plt.tight_layout()
        plt.show()
    else:
        plt.savefig(f"{fname}_cross_time{tstep:04}.png",dpi=300, 
                bbox_inches='tight')

    fig.clear()
    plt.close(fig)
    plt.clf()

    return

color = cmocean.cm.speed
fname = '1km_lat11'
xmin = lon.min() 
xmax = 144.6
size = (9,2)

dmin = 0.
dmax = 1.5

plot_cross_section(0, lon, currLat, xmin, xmax, dmin, dmax, color, size, fname, show=True)

# Looping over time step

The AIMS server for the GBR 1k model only provide daily files. If one want to look at 1 month interval we will have to open each file individually...

In [ ]:
color = cmocean.cm.speed
fname = '1km_lat11'
xmin = lon.min() 
xmax = 144.6
size = (9,2)

dmin = 0.
dmax = 1.5

selectedVariable = 'mean_cur' 

In [ ]:
# Define starting and ending date of the netcdf file we want to load 
start_date = dt.date(2020, 3, 1)
end_date = dt.date(2020, 4, 1)
delta = relativedelta(days=+1)
step = 0

# Now perform a while loop to open the netcdf file and extract the relevant dataset for the site of interest
while start_date <= end_date:
    
    # Read individual file from the OpeNDAP server
    netCDF_datestr = str(start_date.year)+'-'+format(start_date.month, '02')+'-'+format(start_date.day, '02')
    print('Processing time interval:',netCDF_datestr)
    
    inputFile = "http://thredds.ereefs.aims.gov.au/thredds/dodsC/s3://aims-ereefs-public-prod/derived/ncaggregate/ereefs/gbr1_2.0/daily-daily/EREEFS_AIMS-CSIRO_gbr1_2.0_hydro_daily-daily-"+netCDF_datestr+".nc"
    start_date += delta    
    nc_data = Dataset(inputFile, 'r')
    ncdata = nc_data.variables
    
    dataLat = eReefs_cross_data(selectedVariable, selectedLatIndex, None)
    plot_cross_section(step, lon, dataLat, xmin, xmax, dmin, dmax, color, size, fname, show=False)
    step += 1

We can now create an animated gif for our 1km resolution cross section:

In [ ]:
!convert -delay 100 1km_lat11_cross_time*.png 1km_Lat11_vel.gif

In [ ]:
display(HTML("<img src='1km_Lat11_vel.gif' />"))